## 1.3 Introduction to Information Retrieval

Here we work with a data set scraped from eBay.  The data contains 9895 item titles and descriptions.

First we load the data file and _normalise_ the text - removing certain characters and converting to lower case.  

In [ ]:

import pandas as pd
import math

from scipy.sparse import csr_matrix

from bokeh.plotting import figure, output_notebook, show, vplot
from bokeh.charts import Bar, Scatter, BoxPlot
from bokeh.charts.attributes import CatAttr
from bokeh.models import ColumnDataSource

from sys import getsizeof

In [ ]:
import csv
import re

with open("data/bike-items.txt") as f:
    r = csv.reader(f, delimiter=',', quotechar='"')
    rgx = re.compile(r'\b[a-zA-Z]+\b') 
    docs = [ (' '.join(re.findall(rgx, x[0])).lower(), ' '.join(re.findall(rgx, x[1])).lower())  \
                for i,x in enumerate(r) if i > 1 ]
    
print(docs[0][0],docs[0][1])

items_t = [ d[0] for d in docs ] # item titles
items_d = [ d[1] for d in docs ] # item descriptions
items_i = range(0, len(items_t)) # item id


## Exercise Set 1 - Term Frequency

Let's start with the first 10 item titles as out corpus:

In [ ]:
corpus = items_t[0:5]
print(corpus)

In [11]:
!ls resources/

ic_assignment_black_24dp_2x.png  ic_info_outline_black_24dp_2x.png
ic_build_black_24dp_2x.png


Our goal is to enumerate over the corpus of documents, tokenize the documents and count the frequency of tokens. 

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Here is a part completed code snippet to compute term frequency.  
Complete this code to correctly populate the dictionary.

In [ ]:
tf = {}
for doc in corpus:
    for word in doc.split():
        # << CODE HERE
        
print(tf)

We can simplify by using a [Counter](https://docs.python.org/2/library/collections.html#collections.Counter) rather than a dictionary.

<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'> 
Take a look at the docs for the `Counter` collection.  
Complete this code snippet to compute term frequency using the `Counter`.

In [ ]:
from collections import Counter
tf = Counter()
for doc in corpus:
    for word in doc.split():
        # << CODE HERE

print(tf)

The Counter does not give us a real speed advantage - since it does more work.   For these tiny data sets we do not see any difference - however in Python 3 it is faster than a default dictionary.  Often times best way to test performance is to time code execution.

<img src='files/resources/ic_info_outline_black_24dp_2x.png' align='left'> 
We should get used to thinking about performance.   
We can use the Jupyter Notebook [magics](http://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb) to time the execution.

In [9]:
from collections import Counter

def tf1(corpus):
    tf = {}
    # << DICTIONARY TERM FREQUENCY CODE HERE
    return tf

def tf2(corpus):
    tf = Counter()
    # << COUNTER TERM FREQUENCY CODE HERE
    return tf

%timeit tf1(corpus)
%timeit tf2(corpus)


The slowest run took 23.54 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 91.2 ns per loop
The slowest run took 16.05 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 995 ns per loop


<img src='files/resources/ic_assignment_black_24dp_2x.png' align='left'>
Now compute term frequency for the full corpus of item titles.  
What is the frequency of the terms 'Unicycle', 'Bicycle' and 'Tricycle'?

### TF Matrix

Whilst the TF dictionary is a compact way to store the term frequency it is not much use for analysis.  We need a TF matrix where each document vector is the same length.  Now we convert the dictionary to a matrix:

In [ ]:
def get_lexicon(corpus):
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc.split()])
    return lexicon

lexicon = get_lexicon(corpus)

tfm =[]
for doc in corpus:
    for term in doc.split():
        tfv = [doc.split().count(word) for word in lexicon]
    tfm.append(tfv)
        
print([ x for x in tfm])

As number of terms increases this method becomes inefficient.  Here is a faster implementation:

In [ ]:
def get_lexicon(corpus):
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc.split()])
    return list(lexicon)

lexicon = get_lexicon(corpus)

tfm =[]
for doc in corpus:
    tfv = [0]*len(lexicon)
    for term in doc.split():
        tfv[lexicon.index(term)] += 1
    tfm.append(tfv)
        
print(tfm)

We can compare time for each:

In [ ]:
def tfm1(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return lexicon
    
    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc in corpus:
        for term in doc.split():
            tfv = [doc.split().count(word) for word in lexicon]
        tfm.append(tfv)
    
    return tfm, lexicon

def tfm2(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return list(lexicon)

    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc in corpus:
        tfv = [0]*len(lexicon)
        for term in doc.split():
            tfv[lexicon.index(term)] += 1
        tfm.append(tfv)
    
    return (tfm, lexicon)

%timeit tfm1(corpus)
%timeit tfm2(corpus)


In [ ]:
# as size of corpus increases so does the sparsity

n = []
s = []
for i in range(100,1000,100):
    corpus = items_t[0:i]
    tfm, lexicon = tfm2(corpus)
    c =[ [x.count(0), x.count(1)] for x in tfm]
    n_zero = sum([ y[0] for y in c])
    n_one = sum([ y[1] for y in c])  
    s.append(1.0 - (float(n_one) / (n_one + n_zero)))
    n.append(i)
    
output_notebook(hide_banner=True)
p = figure(x_axis_label='Documents', y_axis_label='Sparsity',
          plot_width=400, plot_height=400)
p.line(n, s, line_width=2)
p.circle(n, s, fill_color="white", size=8)
show(p)


We can take advantage of the sparsity and only store the non-zero elements of the TF matrix.

### Spare matrix storage

In [ ]:
def tfm3(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return list(lexicon)

    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc_id, doc in enumerate(corpus):
        tfv = [0]*len(lexicon)
        for term in doc.split():
            tfv[lexicon.index(term)] += 1
        tfm.append([[(doc_id, t_id), t] for t_id, t in enumerate(tfv) if t > 0])
    
    return (tfm, lexicon)

tfm, lexicon = tfm3(corpus)
print(tfm[0])

We can also use compression to store this data even more efficiently - scikit-learn provides a compressed sparse matrix:

In [ ]:
tfm=csr_matrix(tfm2(corpus)[0])
print(tfm[0,:])

l = ['tf2','tfm2','csr']
s = [getsizeof(tf2(corpus)[0]) , getsizeof(tfm2(corpus)[0]), getsizeof(csr_matrix(tfm2(corpus)[0]))]

df = pd.DataFrame({'Type':l, 'Size':s})

output_notebook(hide_banner=True)
p = Bar(df.sort_values(by='Size'), label='Type', values='Size',
        plot_width=400, plot_height=400)
show(p)

## Boolean Search

Now we have a tf matrix we can start to use it to find documents that contain words included in a query.  We will start by simply returning the documents from the corpus that match terms in our query and rank by raw term frequency:

In [ ]:
def get_results1(qry, tfm, lexicon):
    qrv = [0]*len(lexicon)
    for term in qry.split():
        if term in lexicon:
            qrv[lexicon.index(term)] = 1

    results = []      
    for i, tfv in enumerate(tfm):
        score = sum([x[0] * x[1] for x in zip(tfv, qrv)])
        if score > 0:
               results.append([score, i])
    return results

def print_results(results,n, head=True):
    if head:    
        print('\nTop %d from recall set of %d items ordered by tf-idf:' % (n,len(results)))
        for r in sorted(results, key=lambda t: t[0] * -1 )[:n]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    else:
        print('\nBottom %d from recall set of %d items ordered by tf-idf:' % (n,len(results)))
        for r in sorted(results, key=lambda t: t[0] * 1 )[:n]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    
tfm, lexicon = tfm2(items_t)
results = get_results1('front rear back led bike light', tfm , lexicon)

print_results(results,10)


But this is an expensive operation.  Each query has to be compared to all documents in the corpus.  We can speed this up by creating an inverted index:

In [ ]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            if word in idx:
                idx[word].append(i)
            else:
                idx[word] = [i]
    return idx
            
idx = create_inverted_index(items_t)

print(set(idx['front']).intersection(set(idx['rear'])))
print(items_t[7676])


Now we just have to query for each of the terms and produce a set of results:

In [ ]:
def get_results2(qry, idx):

    score = Counter()
    terms = qry.split()
    for term in terms:
        for doc in idx[term]:
            score[doc] += 1
            
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;


idx = create_inverted_index(items_t)
%timeit results = get_results2('front rear back led bike light', idx)

print_results(results,10)


We get a lot of documents in the recall set since many match on one of the words - bike is present in almost every other document!

In [ ]:
df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()]})

output_notebook(hide_banner=True)
p = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
show(p)


## Inverse Document Frequency (IDF)

It would seem sensible to down weight words that are very common in the corpus - the word 'bike' in a query is not as discriminating as the word 'front'. IDF is a way to quantify how common or rare a term is in the corpus.  It is computed by taking the log of the inverse fraction of the number of documents in which the term appears divided by the total number of documents.  To avoid division by zero it is common to add 1 to the number of documents in which the term appears.  

IDF is already partially computed when we constructed the inverted index - it is the number of documents the term apears in - in otherwords the length of the document list in the inverted index

In [ ]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            if word in idx:
                if i in idx[word]:
                    # Update document's frequency
                    idx[word][i] += 1
                else:
                    # Add document
                    idx[word][i] = 1
            else:
                # Add term
                idx[word] = {i:1}
    return idx

def idf(term, idx, n):
    return math.log( float(n) / (1 + len(idx[term])))

idx = create_inverted_index(items_t)


df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()],
                  'idf':[idf(x, idx, len(items_t)) for x in idx.keys()]})

output_notebook(hide_banner=True)
p1 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)

p2 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='idf',
        plot_width=800, plot_height=400)

p = vplot(p1, p2)

show(p)

## Ranking by TF-IDF

We can now combine term frequency and inverse document frequency when computing the score for each item in the recall set.  Until now we have just computed the score as the raw frequency of the query terms in each document.  Now we want to weight the raw frequency by the inverse documetn frequency.

In [ ]:
def get_results3(qry, idx, n):
    score = Counter()
    for term in qry.split():
        if term in idf:
            i = idf(term, idx, n)
            for doc in idx[term]:
                score[doc] += idx[term][doc] * i
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;

In [ ]:
idx = create_inverted_index(items_t)
results = get_results3('front led bike light', idx, len(items_t))

print_results(results,10)

## Problematic queries!

With this corpus we cannot search for mountain bikes without returning a heap of accesories:

In [ ]:
idx = create_inverted_index(items_t)
results = get_results3('mountain bike', idx, len(items_t))

print_results(results,10)

We need to penalise items where there are many more terms in the query.  For example the terms "mountain" and "bike" only make up 2 / 12 terms in the "oakley mens automatic mountain mtb factory lite mountain bmx bike gloves large" yet it scores highly because there is no penalty for all the other terms in the item title.

In addition this scheme create discrete levels based on combination of word frequency:


In [ ]:
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'title':[items_t[x[1]] for x in results]})

d = df.groupby('score').first().reset_index()

r1 = re.compile('(bike)')
r2 = re.compile('(mountain)')

for i, t in enumerate(d.title):
    n1 = r1.findall(t)
    n2 = r2.findall(t)
    print('%d x Bike, %d x Mountain, Score = %0.2f'%(len(n1),len(n2),d.score[i]))
    

In [ ]:
# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook(hide_banner=True)
p = Scatter(df, x='score', y='length')
show(p)

Ideally we do not want scores to be the same for lots of documents.  We could try by boosting the score for documents that are shorter than average.

In [ ]:
def get_results4(qry, corpus):
    idx = create_inverted_index(corpus)
    n = len(corpus)
    d = [len(x.split()) for x in corpus]
    d_avg = float(sum(d)) / len(d)
    score = Counter()
    for term in qry.split():
        if term in idf:
            i = idf(term, idx, n)
            for doc in idx[term]:
                f = float(idx[term][doc])
                score[doc] += i *  ( f / (float(d[doc]) / d_avg) )
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;

In [ ]:
results = get_results4('mountain bike', items_t)
print_results(results, 10)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook()
p = Scatter(df, x='score', y='length')
show(p)

## Okapi BM25

In [ ]:
def get_results5(qry, corpus, k1=1.5, b=0.75):
    idx = create_inverted_index(corpus)
    n = len(corpus)
    d = [len(x.split()) for x in corpus]
    d_avg = float(sum(d)) / len(d)                
    score = Counter()
    for term in qry.split():
        if term in idf:
            i = idf(term, idx, n)
            for doc in idx[term]:
                f = float(idx[term][doc])
                score[doc] += i * (( f * (k1 + 1) ) / (f + k1 * (1 - b + (b * (float(d[doc]) / d_avg)))))
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;

In [ ]:
results = get_results5('mountain bike', items_t)
print_results(results, 10)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t[x[1]].split()) for x in results]})

output_notebook()
p = Scatter(df, x='score', y='length')
show(p)
    

In [ ]:
## Can we make the above interactive?
